In [1]:
function x(k)
    return cos(k*pi/N)
end

x (generic function with 1 method)

In [12]:
function build_c()
    
    c = ones(N+1)
    c[1] = 2
    c[N+1] = 2

    return c
end

build_c (generic function with 2 methods)

In [13]:
function build_D(N)
    c = build_c()
    
    D = zeros(N+1, N+1)
    D[1,1] = (2*N^2+1)/6
    D[N+1, N+1] = -(2*N^2+1)/6
    
    for i = 1:N-1
        D[i+1,i+1] = -1/2*(x(i)/(1-x(i)^2))
    end
    
    for i = 0:N, j = 0:N
        if i != j
            D[i+1,j+1] = c[i+1]*(-1)^(i+j)/(c[j+1]*(x(i) - x(j)))
        end
    end

    return D
end

build_D (generic function with 1 method)

In [14]:
function build_M(N, tau, A0, A1)
    
    D = build_D(N)
    
    tau = 0.02
    C = -2*D/tau
    
    C_hat = C[1:N, 1:N+1]
    
    n, null = size(A0)
    eye = Matrix(1.0I, n, n)

    M_temp = kronecker(C_hat, eye)
    
    sec_row = zeros(n, n*(N+1))
    sec_row[1:n, 1:n] = A1
    sec_row[1:n, N*n + 1:(N+1)*n] = A0

    M = [M_temp
             sec_row]
    return M, n
end

build_M (generic function with 1 method)

In [15]:
# Benchmarking from Milano

N = 1

D1 = [0.5 -0.5
        0.5 -0.5]

D1_our = build_D(N)

@assert D1 ≈ D1_our

In [16]:
N = 2

D2 = [1.5 -2 0.5
        0.5 0 -0.5
            -0.5 2 -1.5]

D2_our = build_D(N)

@assert D2 ≈ D2_our

In [18]:
using Kronecker
using LinearAlgebra

N = 7

A0 = [-2 0 
        0 -1]

A1 = [0 0 
        0 -0.1]

A2 = [-2 0 0
        0 -1 0
         0 0 -3]

A3 = [0 0 0
        0 0 0 
         0 0 -0.1]

# Real matrices from Jia paper

A4 = zeros(4, 4)
A4[1, 2] = 376.9911
A4[2, 1:3] = [-0.0963 -0.5 -0.0801]
A4[3, :] = [-0.0480 0 -0.1667 0.1]
A4[4, 4] = -1

A5 = zeros(4,4)
A5[4, 1] = 38.0187
A5[4, 3] = -95.2560

tau = 0.01

M, n = build_M(N, tau, A0, A1)



([-1650.0 -0.0 … 50.0 0.0; -0.0 -1650.0 … 0.0 50.0; … ; 0.0 0.0 … -2.0 0.0; 0.0 -0.1 … 0.0 -1.0], 2)

In [19]:
println("According to Milano, size of M should be N*n x N*n")

s, t = size(M)
println("measured size of M = $s x $t")
measured_N = s/n
prod = measured_N*n

println("N = $N")
println("n = $n")
println("measured_N = $measured_N")
println("measured_N*n = $prod")
println("measured_N*n x measured_N*n = $(measured_N*n) x $(measured_N*n)")


According to Milano, size of M should be N*n x N*n
measured size of M = 16 x 16
N = 7
n = 2
measured_N = 8.0
measured_N*n = 16.0
measured_N*n x measured_N*n = 16.0 x 16.0


In [20]:
size(M)

(16, 16)

In [21]:
eigs = eigvals(M)

16-element Vector{ComplexF64}:
  -332.8866963853495 + 0.0im
  -326.6925313026702 + 0.0im
  -312.6754369714155 - 166.41401274431607im
  -312.6754369714155 + 166.41401274431607im
  -308.7974772789299 - 166.43748203315153im
  -308.7974772789299 + 166.43748203315153im
  -232.9845353611487 - 345.3253547738202im
  -232.9845353611487 + 345.3253547738202im
 -232.11021225832025 - 345.2134650840502im
 -232.11021225832025 + 345.2134650840502im
 -116.81697065113974 - 557.2615512465941im
 -116.81697065113974 + 557.2615512465941im
 -116.77463916724722 - 557.2070820475063im
 -116.77463916724722 + 557.2070820475063im
                -2.0 + 0.0im
 -1.1022289355797015 + 0.0im

In [22]:
println(eigvals(A0))
println(eigvals(A1))

[-2.0, -1.0]
[-0.1, 0.0]
